## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-06-49-38 +0000,bbc,Qatar hosts Arab-Islamic emergency summit over...,https://www.bbc.com/news/articles/c4gk8nx5nw3o...
1,2025-09-15-06-35-58 +0000,nypost,UK police say 49-year-old German remains a sus...,https://nypost.com/2025/09/15/world-news/uk-po...
2,2025-09-15-06-24-44 +0000,bbc,UK and US unveil nuclear energy deal promising...,https://www.bbc.com/news/articles/ckgzevzwxwro...
3,2025-09-15-06-21-00 +0000,wsj,"BOE Expected to Leave Key Rate on Hold, But Sl...",https://www.wsj.com/world/uk/boe-expected-to-l...
4,2025-09-15-06-03-05 +0000,bbc,Aldi warns food prices may rise if Budget lift...,https://www.bbc.com/news/articles/c0r0zr7ed9yo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
167,kirk,22
305,trump,21
166,charlie,17
14,suspect,9
111,up,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
120,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,86
138,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,80
49,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,74
93,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,72
61,2025-09-14-22-55-24 +0000,nypost,"2 men arrested for allegedly planting, lightin...",https://nypost.com/2025/09/14/us-news/2-men-ar...,70


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
120,86,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
170,36,2025-09-14-09-30-00 +0000,wsj,In the weeks since President Trump’s 50% tarif...,https://www.wsj.com/economy/trade/global-trade...
144,32,2025-09-14-13-00-00 +0000,wsj,"As America’s 250th birthday approaches, Robert...",https://www.wsj.com/business/robert-reed-churc...
9,30,2025-09-15-04-43-10 +0000,nypost,Maniac NYC driver who mowed down teen girl aft...,https://nypost.com/2025/09/15/us-news/maniac-n...
33,30,2025-09-15-02-00-00 +0000,wsj,A bullet casing found after the killing of Cha...,https://www.wsj.com/politics/policy/fascist-en...
20,29,2025-09-15-03-08-00 +0000,wsj,Signs of weakness in China’s economy stretched...,https://www.wsj.com/economy/chinas-economy-sho...
1,27,2025-09-15-06-35-58 +0000,nypost,UK police say 49-year-old German remains a sus...,https://nypost.com/2025/09/15/world-news/uk-po...
53,23,2025-09-14-23-38-15 +0000,nyt,Protests Against an Israeli Cycling Team Shut ...,https://www.nytimes.com/2025/09/14/world/canad...
62,23,2025-09-14-22-49-19 +0000,nypost,Roaming pack of wild dogs — as many as 40 — te...,https://nypost.com/2025/09/14/us-news/roaming-...
88,23,2025-09-14-19-55-03 +0000,nypost,Bank of New York claims Condé Nast space at On...,https://nypost.com/2025/09/14/business/bank-of...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
